# 策略

找到30天内涨幅在80%以上的牛股，当其30日WR回落到0.7以下，发出信号

In [1]:
from alpha.notebook import *
await init_notebook(use_omicron=True)
stats = Stats("rsi")
stats.load("/Users/aaronyang/data/rsi_1d_20190104_20210601.hist")

NameError: name 'Stats' is not defined

In [ ]:
async def check(code, close, adv=0.8, rsi_threshold=30):
    cmax = np.max(close)
    cmin = np.min(close)

    # 忽略一路下降的走势
    if np.argmax(close) < np.argmin(close):
        return None

    max_adv = cmax / cmin - 1
    wr = (close[-1] - cmin) / (cmax - cmin)
    rsi = relative_strength_index(close, 6)[-1]

    if max_adv > adv and rsi < 30:
        prob = stats.query(code, rsi)
        return max_adv, wr, rsi, prob
    
    return None

async def scan(tm, adv=0.8, n=30, df=True):
    end = arrow.get(tm).date()
    start = tf.day_shift(end, -n+1)
    results = []
    for code in Securities().choose(["stock"]):
        sec = Security(code)
        try:
            bars = await sec.load_bars(start, end, FrameType.DAY)
            close = bars["close"]

            if len(close) != n or np.count_nonzero(np.isfinite(close)) < n * 0.9:
                continue

            close = fillna(close.copy())
            result = await check(code, close, adv)
            if result is None:
                continue


            results.append([sec.display_name, code, end, *result])
        except Exception:
            pass
    if df:
        return pd.DataFrame(results, columns=["name", "code", "date", "adv", "wr", "rsi", "prob"])
    else:
        return results

In [ ]:
async def backtest(start, end):
    start = arrow.get(start).date()
    end = arrow.get(end).date()
    results = []
    
    if tf.count_day_frames(end, arrow.now()) < 10:
        end = tf.day_shift(arrow.now(), -10)
        
    for dt in tf.get_frames(start, end, FrameType.DAY):
        signals = await scan(tf.int2date(dt), df=False)
        for name, code, fire_date, adv, wr, rsi, prob in signals:
            tail = tf.day_shift(fire_date, 10)
            try:
                ybars = await Security(code).load_bars(fire_date, tail, FrameType.DAY)
            except Exception:
                continue

            profit, ind = sell_on_stop_lose(ybars["close"])
            sell_date = ybars["frame"][ind]
            duration = tf.count_day_frames(fire_date, sell_date)
            result = [name, code, fire_date, sell_date, duration, profit, adv, wr, rsi, prob]
            results.append(result)
    return results

results = await backtest("2021-09-01", "2021-09-30")

In [ ]:
columns = ["name", "code", "fire_date","sell_date", "duration", "profit", "adv", "wr", "rsi", "prob"]
df = pd.DataFrame(results, columns=columns)

In [31]:
bars = jq_get_bars("301051.XSHE", 12, '1d', '2021-10-21')
close = bars["close"]

In [33]:
d1 = close[1:] / close[:-1] - 1
d2 = np.diff(d1)

dma2 = moving_average(d2, 5)

In [34]:
dma2

array([ 0.00989181, -0.0028264 , -0.0003779 ,  0.00540355,  0.02031308,
       -0.00398905])

In [37]:
d1*100

array([-2.98646757,  0.5964406 , -0.37295591,  0.2015742 , -2.2224351 ,
        1.9594396 , -0.81675795, -0.56190661,  2.90335152,  7.93410339,
       -0.03508772])

In [39]:
d2

array([ 0.03582908, -0.00969397,  0.0057453 , -0.02424009,  0.04181875,
       -0.02776198,  0.00254851,  0.03465258,  0.05030752, -0.07969191])

In [40]:
moving_average(d1, 5)

array([-0.00956769,  0.00032413, -0.00250227, -0.00288017,  0.00252338,
        0.02283646,  0.01884741])

In [42]:
start = arrow.get("2021-10-21")
end = arrow.get("2021-10-21")

tf.get_frames(start, end, FrameType.DAY)

array([20211021])